In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep
import os
import psutil
import inspect

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True,
                           queue_port=3322)
                           
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Object in Action Store that needs migration:  [<class 'syft.service.action.action_object.AnyActionObject'>]
Data Migrated to latest version !!!
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
x = ActionObject.from_obj([1,2])
x_ptr = x.send(client)

In [4]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

In [6]:
@syft_function()
def middle_middle_job(domain, batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    batch_job = domain.launch_job(process_batch, batch=batch)
    print("start leaf job")
    return 2
    

SyftSuccess: Syft function 'middle_middle_job' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [7]:
client.code.submit(middle_middle_job)

SyftSuccess: User Code Submitted

In [8]:
@syft_function()
def middle_job(domain, batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    batch_job = domain.launch_job(middle_middle_job, batch=batch)
    print("start leaf job")
    return 2
    

SyftSuccess: Syft function 'middle_job' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [9]:
client.code.submit(middle_job)

SyftSuccess: User Code Submitted

In [10]:
@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        batch_job = domain.launch_job(middle_job, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    return 1


SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [11]:
req = client.code.request_code_execution(process_all)
req

syft.service.request.request.Request

In [12]:
client.requests[0]

syft.service.request.request.Request

In [13]:
client.requests[0].code

```python
class UserCode
    id: UID = 7c1ee8a6940a4a53a77e5da2cb4f712f
    service_func_name: str = process_all
    shareholders: list = ['test-domain-helm2']
    status: list = ['Node: test-domain-helm2, Status: pending']
    
    code:

@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        batch_job = domain.launch_job(middle_job, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    return 1



  Nested Requests:
  class UserCode
      id: UID = 88b518c3b9094eaf99915a6ba486c5f1
      service_func_name: str = middle_job
      shareholders: list = []
      status: list = ['Node: test-domain-helm2, Status: pending']
      
      code:
  
  @syft_function()
  def middle_job(domain, batch):
      # takes 30 hours normally
      print(f"starting batch {batch}")
      from time import sleep
      sleep(1)
      batch_job = domain.launch_job(middle_middle_job, batch=batch)
      print("start leaf job")
      return 2
  
  
  
    Nested Requests:
    class UserCode
        id: UID = e0b1e4df56124db094f1e49c92c2c926
        service_func_name: str = middle_middle_job
        shareholders: list = []
        status: list = ['Node: test-domain-helm2, Status: pending']
        
        code:
    
    @syft_function()
    def middle_middle_job(domain, batch):
        # takes 30 hours normally
        print(f"starting batch {batch}")
        from time import sleep
        sleep(1)
        batch_job = domain.launch_job(process_batch, batch=batch)
        print("start leaf job")
        return 2
    
    
    
      Nested Requests:
      class UserCode
          id: UID = 4536050e4f7440e69f0abf0289f31f40
          service_func_name: str = process_batch
          shareholders: list = []
          status: list = ['Node: test-domain-helm2, Status: pending']
          
          code:
      
      @syft_function()
      def process_batch(batch):
          # takes 30 hours normally
          print(f"starting batch {batch}")
          from time import sleep
          sleep(1)
          print("done")
          return batch+1
      
```

In [14]:
client.requests[0].codes

[syft.service.code.user_code.UserCode,
 syft.service.code.user_code.UserCode,
 syft.service.code.user_code.UserCode,
 syft.service.code.user_code.UserCode]

In [15]:
### Deciding if to approve the requests....

In [34]:
client.requests[-1].code

```python
class UserCode
    id: UID = 7c1ee8a6940a4a53a77e5da2cb4f712f
    service_func_name: str = process_all
    shareholders: list = ['test-domain-helm2']
    status: list = ['Node: test-domain-helm2, Status: approved']
    
    code:

@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        batch_job = domain.launch_job(middle_job, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    return 1



  Nested Requests:
  class UserCode
      id: UID = 88b518c3b9094eaf99915a6ba486c5f1
      service_func_name: str = middle_job
      shareholders: list = []
      status: list = ['Node: test-domain-helm2, Status: pending']
      
      code:
  
  @syft_function()
  def middle_job(domain, batch):
      # takes 30 hours normally
      print(f"starting batch {batch}")
      from time import sleep
      sleep(1)
      batch_job = domain.launch_job(middle_middle_job, batch=batch)
      print("start leaf job")
      return 2
  
  
  
    Nested Requests:
    class UserCode
        id: UID = e0b1e4df56124db094f1e49c92c2c926
        service_func_name: str = middle_middle_job
        shareholders: list = []
        status: list = ['Node: test-domain-helm2, Status: pending']
        
        code:
    
    @syft_function()
    def middle_middle_job(domain, batch):
        # takes 30 hours normally
        print(f"starting batch {batch}")
        from time import sleep
        sleep(1)
        batch_job = domain.launch_job(process_batch, batch=batch)
        print("start leaf job")
        return 2
    
    
    
      Nested Requests:
      class UserCode
          id: UID = 4536050e4f7440e69f0abf0289f31f40
          service_func_name: str = process_batch
          shareholders: list = []
          status: list = ['Node: test-domain-helm2, Status: pending']
          
          code:
      
      @syft_function()
      def process_batch(batch):
          # takes 30 hours normally
          print(f"starting batch {batch}")
          from time import sleep
          sleep(1)
          print("done")
          return batch+1
      
```

In [17]:
client.requests[-1].approve(approve_nested=True)

Request approved for domain test-domain-helm2


SyftSuccess: Request 11df41e55c62490c8d69e3203ada2d50 changes applied

In [18]:
client.code

In [19]:
client.code.process_all

<function syft.client.api.generate_remote_function.<locals>.wrapper(*args, **kwargs)>

In [20]:
code = client.code[3]

In [21]:
client = node.login(email="info@openmined.org", password="changethis")

Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [22]:
client.code.process_all

<function syft.client.api.generate_remote_function.<locals>.wrapper(*args, **kwargs)>

In [23]:
client.code.process_all()

Err("process_all() missing required keyword argument: 'x'")


SyftError: Failed to run. 'Err' object has no attribute 'id', Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code_service.py", line 330, in _call
    output_result = action_service.set_result_to_store(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/action/action_service.py", line 275, in set_result_to_store
    result_id = result_action_object.id
                ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Err' object has no attribute 'id'

In [24]:
client.api.services.code.call(uid=code.id, x=x_ptr)

Launching jobs


Exception in thread Thread-5 (read_items):
Traceback (most recent call last):
  File "/home/teo/anaconda3/envs/syft_3.11/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/teo/anaconda3/envs/syft_3.11/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/queue/zmq_queue.py", line 126, in read_items
    for item in items:
TypeError: 'NoneType' object is not iterable


Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
Timeout elapsed after 30 seconds while trying to acquiring lock.
FAILED TO LOCK
starting aggregation
Done
1


Pointer:
None

In [25]:
job = client.code.process_all(x=x_ptr, blocking=False)

In [26]:
job

```python
class Job:
    id: UID = 278a48cdaf99450587859ae5cc7afe30
    status: JobStatus.CREATED
    has_parent: False
    result: None
    logs:

0 
    
```

In [27]:
job.subjobs[0]

IndexError: list index out of range

In [ ]:
[[[subjob.result.get() for subjob in job.subjobs] for job in job.subjobs] for job in job.subjobs]

NameError: name 'job' is not defined